# Complex Types

Explore built-in functions for working with collections and strings.

### Objectives
1. Apply collection functions to process arrays
2. Union DataFrames together

### Methods
- [DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html): `union`, `unionByName`
- [Built-In Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html):
  - Aggregate: `collect_set`
  - Collection: `array_contains`, `element_at`, `explode`
  - String: `split`


### Generate inline CSV and load it into DataFrame

In [ ]:
import pandas as pd

data = {
    "email": ["a@example.com", "b@example.com", "a@example.com", "c@example.com"],
    "items": [
        [{"item_name": "Twin Mattress"}],
        [{"item_name": "Queen Mattress"}],
        [{"item_name": "Full Mattress"}],
        [{"item_name": "Pillow"}]
    ]
}

pdf = pd.DataFrame(data)
csv_path = "/dbfs/tmp/sales_data.json"
pdf.to_json(csv_path, orient="records", lines=True)

# Load to Spark
df = spark.read.option("multiline", "false").json(csv_path)
display(df)

In [ ]:
from pyspark.sql.functions import explode, split, col

details_df = (df
              .withColumn("items", explode("items"))
              .select("email", "items.item_name")
              .withColumn("details", split(col("item_name"), " "))
             )
display(details_df)

### String Functions

| Method | Description |
|--------|-------------|
| translate | Translate characters |
| regexp_replace | Regex-based replacement |
| regexp_extract | Extract regex group |
| ltrim | Trim leading spaces |
| lower | Convert to lowercase |
| split | Split a string |


In [ ]:
from pyspark.sql.functions import split
display(df.select(split(df.email, '@')[0].alias('email_handle')))

### Collection Functions

| Method | Description |
|--------|-------------|
| array_contains | Check if array contains element |
| element_at | Return element at given index (1-based) |
| explode | One row per element |
| collect_set | Unique items in group |


In [ ]:
from pyspark.sql.functions import array_contains, element_at

mattress_df = (details_df
               .filter(array_contains(col("details"), "Mattress"))
               .withColumn("size", element_at(col("details"), 1)))
display(mattress_df)

### Aggregate Functions

| Method | Description |
|--------|-------------|
| collect_list | All values in group |
| collect_set | Unique values in group |


In [ ]:
from pyspark.sql.functions import collect_set

size_df = mattress_df.groupBy("email").agg(collect_set("size").alias("size options"))
display(size_df)

## Union and unionByName

> `union` resolves by **position**, `unionByName` resolves by **column name**.
> Neither removes duplicates.

Check if schemas match:


In [ ]:
mattress_df.schema == size_df.schema

In [ ]:
union_count = mattress_df.select("email").union(size_df.select("email")).count()

mattress_count = mattress_df.count()
size_count = size_df.count()

mattress_count + size_count == union_count